<a href="https://colab.research.google.com/github/rajeswarimuppidi/EVA-4-Assignments-5/blob/master/Copy_of_EVA4_Session_5_F4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 10, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(10)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3)
        self.bn5 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10, 20, 3)
        self.bn6 = nn.BatchNorm2d(20)
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.conv7 = nn.Conv2d(20, 10, 1)
        self.dout = nn.Dropout(p=0.05)
           
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.dout(x)
        x = self.pool1(self.bn2(F.relu(self.conv2(x))))
        x = self.dout(x)
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.dout(x)
        x = self.pool2(self.bn4(F.relu(self.conv4(x))))
        x = self.dout(x)
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.dout(x)
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.gap(x)
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 28, 28]             200
       BatchNorm2d-2           [-1, 20, 28, 28]              40
           Dropout-3           [-1, 20, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,896
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
           Dropout-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 10, 14, 14]           1,450
      BatchNorm2d-12           [-1, 10, 14, 14]              20
        MaxPool2d-13             [-1, 10, 7, 7]               0
          Dropout-14             [-1, 1

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Train_Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nEpoch:',epoch)
    print('\nTest set: Average loss: {:.4f}, Test_Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Loss=0.12955664098262787 Batch_id=468 Train_Accuracy=90.71: 100%|██████████| 469/469 [00:19<00:00, 24.64it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 1

Test set: Average loss: 0.0610, Test_Accuracy: 9836/10000 (98.36%)



Loss=0.04419739916920662 Batch_id=468 Train_Accuracy=98.02: 100%|██████████| 469/469 [00:18<00:00, 26.18it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 2

Test set: Average loss: 0.0449, Test_Accuracy: 9874/10000 (98.74%)



Loss=0.009853839874267578 Batch_id=468 Train_Accuracy=98.54: 100%|██████████| 469/469 [00:18<00:00, 26.53it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Epoch: 13

Test set: Average loss: 0.0225, Test_Accuracy: 9932/10000 (99.32%)



Loss=0.017773127183318138 Batch_id=468 Train_Accuracy=99.23: 100%|██████████| 469/469 [00:18<00:00, 26.21it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 14

Test set: Average loss: 0.0212, Test_Accuracy: 9931/10000 (99.31%)



Loss=0.0023959975223988295 Batch_id=468 Train_Accuracy=99.27: 100%|██████████| 469/469 [00:18<00:00, 25.31it/s]



Epoch: 15

Test set: Average loss: 0.0203, Test_Accuracy: 9939/10000 (99.39%)



Target:

As the parameters are reduced add dropout and GAP to the model.

Results:

Parameters: 9K

Best Training Accuracy: 99.27%

Best Test Accuracy: 99.39% (seen consistently in last few epochs)

Analysis:

Model is light weight.

This is a fit model. 

Model can be further improved.


